<a href="https://colab.research.google.com/github/ancestor9/Web_crawling/blob/main/apify_web_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.path.append('..')

In [ ]:
!pip install -q langchain
!pip install -q langchain_community
!pip install -q apify-client

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import langchain
print(langchain.__version__)

ModuleNotFoundError: No module named 'langchain'

In [ ]:
from apify_client import ApifyClient
from langchain.document_loaders.apify_dataset import ApifyDatasetLoader
from config import APIFY_API_TOKEN

# when I tested this, I needed to put the token here. The environment variable wouldn't do:
apify = ApifyClient(
    token=APIFY_API_TOKEN
)
actor = apify.actor(
    actor_id="apify/website-content-crawler",
)

ModuleNotFoundError: No module named 'langchain'

In [ ]:
# initiate run and wait to finish:
client_run = actor.call(
    run_input={"startUrls": [{"url": "https://docs.smith.langchain.com/overview"}]},
)
# https://python.langchain.com/en/latest/

In [ ]:
dataset_id = client_run.get("defaultDatasetId")
print(dataset_id)  # you can also find the ids at https://console.apify.com/storage/datasets

NpCG9x1sGpBoXK1zV


In [ ]:
from langchain.schema import Document

loader = ApifyDatasetLoader(
    dataset_id=dataset_id,
    dataset_mapping_function=lambda dataset_item: Document(
        page_content=dataset_item["text"], metadata={"source": dataset_item["url"]}
    ),
)

In [ ]:
persist_directory = os.path.expanduser("~") + "/Documents/indexes/langsmith"

In [ ]:
persist_directory

'/Users/ben/Documents/indexes/langsmith'

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma

# persist index to disk and load it up again
index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    vectorstore_kwargs={
        "persist_directory": persist_directory
    }).from_loaders([loader])

In [ ]:
# Finally, query the vector database:
query = "What is LangSmith?"
result = index.query_with_sources(query)
print(result["answer"])
print(result["sources"])

 LangSmith is a tool used to debug LLMs, chains, and agents, track token usage, and collect examples of unexpected outcomes.

https://docs.smith.langchain.com/overview


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

# we need embeddings for retrieval
embeddings = OpenAIEmbeddings()

In [ ]:
# load the index back up:
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
from langchain.chains import FlareChain
from langchain.chat_models import ChatOpenAI

LLM = ChatOpenAI(
    model_name="gpt-4",
    temperature=0,
)
agent = FlareChain.from_llm(
    llm=LLM,
    retriever=db.as_retriever(),
    max_generation_len=1500,
    verbose=True,
    max_tokens_limit=1500,
)

In [ ]:
agent.run("What is LangSmith and what can it do?")



> Entering new FlareChain chain...
Current Response: 
Prompt after formatting:
Respond to the user message using any relevant context. If context is provided, you should ground your answer in that context. Once you're done responding return FINISHED.

>>> CONTEXT: 
>>> USER INPUT: What is LangSmith and what can it do?
>>> RESPONSE: 
Generated Questions: ['What are some of the tools and resources that LangSmith offers to aid in language learning?', '"What is one of the features that LangSmith offers to aid in language learning?"', '"What feature does LangSmith provide to help you understand your learning progress and areas for improvement?"']

> Finished chain.


' LangSmith is a platform for building, debugging, and monitoring natural language processing (NLP) applications. It provides a suite of tools for debugging LLMs, chains, agents, tools, and retrievers, as well as for monitoring performance and collecting feedback. It also makes it easy to curate datasets and export them for use in other contexts. '